<a href="https://colab.research.google.com/github/SoumyaAIDev/Hugging_Face_LLM_models/blob/main/Quangn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("Quangnguyen711/solidity_re_entrancy_dataset")
print(dataset["train"].to_pandas().head(5))

README.md:   0%|          | 0.00/454 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.42M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/490k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17505 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1945 [00:00<?, ? examples/s]

                                                func  label  __index_level_0__
0  function increaseApprovalPreSigned(\n        b...      0              10030
1  function addOperator(address newOperator) publ...      0              13932
2  function doBorrowAndSell(\n        MarginState...      0              14240
3  function donateToken(address _token, uint256 _...      1               3826
4  function getFirstRevealedBug(uint256 bountyId)...      0              14341


In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import Dataset

# Load pre-trained T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Example input DataFrame
data = {
    "func": [
        "function increaseApprovalPreSigned(\n        bytes32 hash, address spender) external returns (bool) { ... }",
        "function addOperator(address newOperator) public { ... }",
        "function doBorrowAndSell(\n        MarginState state, address borrower) public { ... }",
        "function donateToken(address _token, uint256 _amount) external { ... }",
        "function getFirstRevealedBug(uint256 bountyId) external view returns (uint256) { ... }"
    ],
    "label": [0, 0, 0, 1, 0],
    "__index_level_0__": [10030, 13932, 14240, 3826, 14341]
}

# Convert DataFrame into target format
df = pd.DataFrame(data)
df['instruction'] = "Analyze the following smart contract function and explain its behavior."
df['source_code'] = df['func']

# Define the dataset in dictionary format
dataset_dict = {
    "instruction": df['instruction'].tolist(),
    "source_code": df['source_code'].tolist()
}

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict(dataset_dict)

# Preprocessing function
def preprocess_function(examples):
    inputs = [f"Instruction: {instr} Code:" for instr in examples["instruction"]]
    targets = examples["source_code"]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Print the tokenized dataset
print(tokenized_dataset)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'source_code', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5
})


In [ ]:
 from transformers import Trainer, TrainingArguments

 tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Split the dataset into training and validation sets
train_dataset = tokenized_dataset.select(range(3))  # First three samples for training
validation_dataset = tokenized_dataset.select(range(3, 5))  # Remaining samples for validation

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",                 # Directory for saving model outputs
    evaluation_strategy="epoch",           # Evaluate after every epoch
    learning_rate=2e-5,                    # Learning rate
    per_device_train_batch_size=8,         # Batch size for training
    per_device_eval_batch_size=8,          # Batch size for evaluation
    num_train_epochs=3,                    # Number of epochs
    weight_decay=0.01,                     # Weight decay to prevent overfitting
    logging_dir="./logs",                  # Directory for logging
    logging_steps=10,                      # Log after every 10 steps
)

# Initialize Trainer
trainer = Trainer(
    model=model,                          # T5 model
    args=training_args,                   # Training arguments
    train_dataset=train_dataset,          # Training dataset
    eval_dataset=validation_dataset       # Validation dataset
)

# Train the model
trainer.train()



Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,10.923475
2,No log,10.466490
3,No log,10.250118


TrainOutput(global_step=3, training_loss=13.672429402669271, metrics={'train_runtime': 112.125, 'train_samples_per_second': 0.08, 'train_steps_per_second': 0.027, 'total_flos': 1218076213248.0, 'train_loss': 13.672429402669271, 'epoch': 3.0})

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)



Evaluation Results: {'eval_loss': 10.250118255615234, 'eval_runtime': 2.9872, 'eval_samples_per_second': 0.67, 'eval_steps_per_second': 0.335, 'epoch': 3.0}


In [ ]:
from datasets import Dataset

# Define the dataset in dictionary format
data = {
    "func": [
        "function increaseApprovalPreSigned(\n        bytes32 hash, address spender) external returns (bool) { ... }",
        "function addOperator(address newOperator) public { ... }",
        "function doBorrowAndSell(\n        MarginState state, address borrower) public { ... }",
        "function donateToken(address _token, uint256 _amount) external { ... }",
        "function getFirstRevealedBug(uint256 bountyId) external view returns (uint256) { ... }"
    ],
    "label": [0, 0, 0, 1, 0],
    "__index_level_0__": [10030, 13932, 14240, 3826, 14341]
}

# Convert the data into a Hugging Face Dataset
dataset = Dataset.from_dict(data)

# Function to search the dataset for a specific query
def search_dataset(dataset, prompt):
    """
    Searches the dataset for functions matching the prompt.

    Args:
        dataset: The Hugging Face dataset.
        prompt: User-provided search query.

    Returns:
        Matching rows (func and label) or a message if no matches are found.
    """
    # Filter rows where the func column contains the prompt (case-insensitive)
    filtered_rows = dataset.filter(lambda example: prompt.lower() in example["func"].lower())

    if len(filtered_rows) == 0:
        return "No matching entries found for the given prompt."

    return filtered_rows

# Prompt input from the user
prompt = input("Enter a function keyword or query to search in the dataset: ")

# Execute the search
results = search_dataset(dataset, prompt)

# Display results
if isinstance(results, str):  # If no matches found
    print(results)
else:
    for idx, row in enumerate(results):
        print(f"Function: {row['func']}")
        print(f"Label: {row['label']}")
        print(f"Index: {row['__index_level_0__']}")
        print("-" * 50)


Enter a function keyword or query to search in the dataset: function setPreferredCurrency(bytes4 currencyKey) external optionalProxy { require(currencyKey == 0 || !exchangeRates.rateIsStale(currencyKey),


Filter:   0%|          | 0/5 [00:00<?, ? examples/s]

No matching entries found for the given prompt.
